<a href="https://colab.research.google.com/github/aetev/Learning-stuff-/blob/main/trading%20sac%20indicator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tf_agents[reverb]
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.4/624.4 kB 45.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 73.5 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.23.0-py3-none-any.whl size=697641 sha256=3da3d49403069fd466b203e4cdc07ab4c56136ad696da944634d0592045ba29b
  Stored in directory: /root/.cache/pip/wheels/3d/6f/b4/3991d4fae11d0ecb0754c11cc1b4e7745012850da4efaaf0b1
Successfully built gym
  Attemptin

In [3]:
import base64
import imageio
import IPython
import matplotlib.pyplot as plt
import os
import reverb
import tempfile
import PIL.Image

import pandas as pd
import numpy as np

import pandas_ta as ta

from tf_agents.environments import suite_gym
import tensorflow as tf
import random

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.trajectories import time_step as ts
from tf_agents.specs import array_spec
from tf_agents.environments import utils

from tf_agents.agents.ddpg import critic_network
from tf_agents.agents.sac import sac_agent
from tf_agents.agents.sac import tanh_normal_projection_network
from tf_agents.metrics import py_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import greedy_policy
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_py_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.train import actor
from tf_agents.train import learner
from tf_agents.train import triggers
from tf_agents.train.utils import spec_utils
from tf_agents.train.utils import strategy_utils
from tf_agents.train.utils import train_utils

RuntimeError: ignored

In [4]:
# Use "num_iterations = 1e6" for better results (2 hrs)
# 1e5 is just so this doesn't take too long (1 hr)
num_iterations = 1000000 # @param {type:"integer"}

initial_collect_steps = 10000 # @param {type:"integer"}
collect_steps_per_iteration = 1 # @param {type:"integer"}
replay_buffer_capacity = 10000 # @param {type:"integer"}

batch_size = 256 # @param {type:"integer"}

critic_learning_rate = 3e-4 # @param {type:"number"}
actor_learning_rate = 3e-4 # @param {type:"number"}
alpha_learning_rate = 3e-4 # @param {type:"number"}
target_update_tau = 0.005 # @param {type:"number"}
target_update_period = 1 # @param {type:"number"}
gamma = 0.99 # @param {type:"number"}
reward_scale_factor = 1.0 # @param {type:"number"}

actor_fc_layer_params = (256, 256)
critic_joint_fc_layer_params = (256, 256)

log_interval = 1000 # @param {type:"integer"}

num_eval_episodes = 5 # @param {type:"integer"}
eval_interval = 10000 # @param {type:"integer"}

policy_save_interval = 5000 # @param {type:"integer"}

In [5]:
def get_relative_change(df, column_name):
    relative_changes = []
    for i in range(len(df)):
        if i == 0:
            relative_changes.append(0)  # First element has no previous value
        else:
            relative_change = (df.iloc[i] - df.iloc[i-1]) / df.iloc[i-1]
            relative_changes.append(relative_change)
    return pd.DataFrame({column_name: relative_changes})


In [6]:
df = pd.read_csv('/content/drive/MyDrive/eurusd_hour.csv')
pd_close = df['BC']
bid_close = pd_close.to_numpy()


change = get_relative_change(pd_close,"Change")
rsi = ta.rsi(pd_close,14,scalar=1)
stdev = ta.stdev(pd_close,14)
macd = ta.macd(pd_close,14)


df = pd.concat([change,rsi,stdev,macd], axis=1)
df = df.dropna(axis=0)
df = df.reset_index()

change = df.pop('Change')
df.pop('index')

indicators = df.to_numpy()
change = change.to_numpy()
change_dev = np.std(change)
np.set_printoptions(suppress=True)
print(change_dev)

0.0012178838054657292


In [7]:
commission_rate = .005
default_val = 10
max_risk = .05
max_ep_len = 5000

In [8]:
class CardGameEnv(py_environment.PyEnvironment):
    def __init__(self):
        self._action_spec = array_spec.BoundedArraySpec(
            shape=(1,), dtype=np.float32, minimum=[-1], maximum=[1], name='action')
        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(7,), dtype=np.float32, minimum=-10, maximum=10, name='observation')

        self._starting_val = random.randint(0,10000)
        self._state = np.append(indicators[self._starting_val],(0,0))
        self._episode_ended = False
        self._step_count = self._starting_val + 1
        self._net_profit = 0



    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec



    def _reset(self):

        self._starting_val = random.randint(0,10000)
        self._state = np.append(indicators[self._starting_val],(0,0))
        self._episode_ended = False
        self._step_count = self._starting_val + 1
        self._net_profit = 0




        return ts.restart(np.array(self._state, dtype=np.float32))



    def _step(self, action):
        if self._episode_ended:
            return self.reset()

        action = action[0]

        cur_change = change[self._step_count]

        cur_val = action * cur_change

        self._net_profit += cur_val

        #update state
        self._state = np.append(indicators[self._step_count],(action,cur_change))

        #end episode if max steps or invalid_action
        if self._step_count >= self._starting_val + max_ep_len:
            self._episode_ended = True

        self._step_count += 1

        if self._episode_ended:
            reward = self._net_profit#self._total_pen
            return ts.termination(np.array(self._state, dtype=np.float32), reward=reward)
        else:
            reward = cur_val
            return ts.transition(np.array(self._state, dtype=np.float32), reward=reward,discount=.5)

In [9]:

environment = CardGameEnv()
utils.validate_py_environment(environment, episodes=2)
#env = environment#tf_py_environment.TFPyEnvironment(environment)

In [10]:
env = environment #tf_py_environment.TFPyEnvironment(environment)
env_name = 'MountainCarContinuous-v0'
collect_env  = env #suite_gym.load(env_name)
eval_env   = env #suite_gym.load(env_name)

In [11]:
use_gpu = False

strategy = strategy_utils.get_strategy(tpu=False, use_gpu=use_gpu)

In [12]:
observation_spec, action_spec, time_step_spec = (
      spec_utils.get_tensor_specs(collect_env))

with strategy.scope():
  critic_net = critic_network.CriticNetwork(
        (observation_spec, action_spec),
        observation_fc_layer_params=None,
        action_fc_layer_params=None,
        joint_fc_layer_params=(20,20),
        kernel_initializer='glorot_uniform',
        last_kernel_initializer='glorot_uniform')

In [13]:
print("Observation Spec:", observation_spec)
print("Action Spec:", action_spec)

Observation Spec: BoundedTensorSpec(shape=(7,), dtype=tf.float32, name='observation', minimum=array(-10., dtype=float32), maximum=array(10., dtype=float32))
Action Spec: BoundedTensorSpec(shape=(1,), dtype=tf.float32, name='action', minimum=array([-1.], dtype=float32), maximum=array([1.], dtype=float32))


In [14]:
with strategy.scope():
  actor_net = actor_distribution_network.ActorDistributionNetwork(
      observation_spec,
      action_spec,
      fc_layer_params=(20,20),
      continuous_projection_net=(
          tanh_normal_projection_network.TanhNormalProjectionNetwork))

In [15]:
with strategy.scope():
  train_step = train_utils.create_train_step()

  tf_agent = sac_agent.SacAgent(
        time_step_spec,
        action_spec,
        actor_network=actor_net,
        critic_network=critic_net,
        actor_optimizer=tf.keras.optimizers.Adam(
            learning_rate=actor_learning_rate),
        critic_optimizer=tf.keras.optimizers.Adam(
            learning_rate=critic_learning_rate),
        alpha_optimizer=tf.keras.optimizers.Adam(
            learning_rate=alpha_learning_rate),
        target_update_tau=target_update_tau,
        target_update_period=target_update_period,
        td_errors_loss_fn=tf.math.squared_difference,
        gamma=gamma,
        reward_scale_factor=reward_scale_factor,
        train_step_counter=train_step)

  tf_agent.initialize()

In [16]:
rate_limiter=reverb.rate_limiters.SampleToInsertRatio(samples_per_insert=3.0, min_size_to_sample=3, error_buffer=3.0)

table_name = 'uniform_table'
table = reverb.Table(
    table_name,
    max_size=replay_buffer_capacity,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1))

reverb_server = reverb.Server([table])

reverb_replay = reverb_replay_buffer.ReverbReplayBuffer(
    tf_agent.collect_data_spec,
    sequence_length=2,
    table_name=table_name,
    local_server=reverb_server)

In [17]:
dataset = reverb_replay.as_dataset(
      sample_batch_size=batch_size, num_steps=2).prefetch(50)
experience_dataset_fn = lambda: dataset

In [18]:
tf_eval_policy = tf_agent.policy
eval_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_eval_policy, use_tf_function=True)

tf_collect_policy = tf_agent.collect_policy
collect_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_collect_policy, use_tf_function=True)

In [19]:
random_policy = random_py_policy.RandomPyPolicy(
  collect_env.time_step_spec(), collect_env.action_spec())

In [20]:
rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  reverb_replay.py_client,
  table_name,
  sequence_length=2,
  stride_length=1)

In [21]:
initial_collect_actor = actor.Actor(
  collect_env,
  random_policy,
  train_step,
  steps_per_run=initial_collect_steps,
  observers=[rb_observer])
initial_collect_actor.run()

In [22]:
location = "/content/drive/MyDrive/Model_foler/Mod1"

In [23]:

env_step_metric = py_metrics.EnvironmentSteps()
collect_actor = actor.Actor(
  collect_env,
  collect_policy,
  train_step,
  steps_per_run=1,
  metrics=actor.collect_metrics(10),
  summary_dir=os.path.join(location, learner.TRAIN_DIR),
  observers=[rb_observer, env_step_metric])

In [24]:
eval_actor = actor.Actor(
  eval_env,
  eval_policy,
  train_step,
  episodes_per_run=num_eval_episodes,
  metrics=actor.eval_metrics(num_eval_episodes),
  summary_dir=os.path.join(location, 'eval'),
)

In [25]:
saved_model_dir = os.path.join(location, learner.POLICY_SAVED_MODEL_DIR)

# Triggers to save the agent's policy checkpoints.
learning_triggers = [
    triggers.PolicySavedModelTrigger(
        saved_model_dir,
        tf_agent,
        train_step,
        interval=policy_save_interval),
    triggers.StepPerSecondLogTrigger(train_step, interval=1000),
]

agent_learner = learner.Learner(
  location,
  train_step,
  tf_agent,
  experience_dataset_fn,
  triggers=learning_triggers,
  strategy=strategy)

Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


In [26]:
def get_eval_metrics():
  eval_actor.run()
  results = {}
  for metric in eval_actor.metrics:
    results[metric.name] = metric.result()
  return results

metrics = get_eval_metrics()

In [27]:
def log_eval_metrics(step, metrics):
  eval_results = (', ').join(
      '{} = {:.6f}'.format(name, result) for name, result in metrics.items())
  print('step = {0}: {1}'.format(step, eval_results))

log_eval_metrics(0, metrics)

step = 0: AverageReturn = -0.009489, AverageEpisodeLength = 5000.000000


In [ ]:
# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = get_eval_metrics()["AverageReturn"]
returns = [avg_return]

for _ in range(num_iterations):
  # Training.
  collect_actor.run()
  loss_info = agent_learner.run(iterations=1)

  # Evaluating.
  step = agent_learner.train_step_numpy

  if eval_interval and step % eval_interval == 0:
    metrics = get_eval_metrics()
    log_eval_metrics(step, metrics)
    returns.append(metrics["AverageReturn"])

  if log_interval and step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, loss_info.loss.numpy()))

rb_observer.close()
reverb_server.stop()

step = 1000: loss = -6.242703437805176
step = 2000: loss = -0.856689989566803
step = 3000: loss = -0.6459925770759583
step = 4000: loss = -1.4494155645370483
step = 5000: loss = -0.8806740045547485
step = 6000: loss = -1.4620229005813599
step = 7000: loss = 0.12397831678390503
step = 8000: loss = 0.6182622909545898
step = 9000: loss = 1.3557088375091553
step = 10000: AverageReturn = 0.072471, AverageEpisodeLength = 5000.200195
step = 10000: loss = -0.9146647453308105
step = 11000: loss = 3.6555144786834717
step = 12000: loss = 0.6765638589859009
step = 13000: loss = 0.26623326539993286
step = 14000: loss = 0.7152742743492126
step = 15000: loss = -2.111241579055786
step = 16000: loss = -2.1701903343200684
step = 17000: loss = 1.2632752656936646
step = 18000: loss = -0.3420027792453766
step = 19000: loss = -1.7012863159179688
step = 20000: AverageReturn = 0.047942, AverageEpisodeLength = 4999.799805
step = 20000: loss = 0.32669904828071594
step = 21000: loss = -1.9018296003341675
step = 